In [13]:
import os
%load_ext autoreload
%autoreload 2
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from logic.data_processing.data_import import dataload
from logic.data_processing.data_processing import data_proc
from logic.ml.classification_shap import IDEARs_funcs

drops=['date_of_attending_assessment_centre_f53_0_0','AD']
wordsremove='inpatient_record|patient_polymorph|time_since_interview|_HES|records_in_hes|treatment_speciality|Diag_PD|Age_Diag_Dementia|Age_Diag_PD|\
Parkinson|interviewer|date_of_attending_assessment_centre_f53|years_after_dis|\
Frontotemporal|daysto|hospital_recoded|from_hospital|Age_Today|year_of_birth|pollution_|pesticide_exposure|\
parental_ad_status_|birth_weight|parkins|sex_inference|\
admission_polymorphic|faster_mot|drive_faster_than|time_to_complete_round|Genotype|genetic_principal|xxxx'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
ac=dataload()
dp=data_proc()
ac=IDEARs_funcs()

In [11]:
%%time
df_dis_date=dp.dis_date_file()

KeyboardInterrupt: 

In [18]:
df=ac.output_treats(load=1)
icd10=ac.ICD10_out()

In [16]:
ratio=0.2
agemin=50
agemax=70
df_pd=pd.read_parquet(ac.path+'df_PD_newcols.parquet')#df_pd_20210924
df_pd.rename(columns={'total_dis':'total ICD10 conditions at baseline'},inplace=True)
mask_age=(df_pd['age_when_attended_assessment_centre_f21003_0_0']>=agemin)&(df_pd['age_when_attended_assessment_centre_f21003_0_0']<=agemax)

df_pd=ac.maskapoedf(df_pd[mask_age],apoe=3)

#for v in agenorm_vars:
#    df_pd=ac.agenorm(df_pd,v)

mask=(df_pd['PD']==1)
df_pd_val=pd.concat([df_pd[mask].sample(round(ratio*df_pd[mask].shape[0])),df_pd[~mask].sample(round(ratio*df_pd[~mask].shape[0]))],axis=0)
mask_val=(df_pd['eid'].isin(df_pd_val['eid']))
df_pd=df_pd[~mask_val]

In [ ]:
%%time
dropsPD=drops+['dementia']
outs_pd=ac.runmodels(df=df_pd,depvar='PD',reps=1,splits=2,drops=dropsPD,wordsremove=wordsremove,model=ac.model(),
             featsfit=30,LRcheck=0,resizeratio=30,agemin=50,agemax=70,verbose=0,apoe=3,tree=1,
             plot_type='dot')

k2=ac.ABS_SHAP(outs_pd[3], outs_pd[4],25,figx=15,figy=15)


Total PD 1468.0
[19:50:33] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { enable_categorical } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [ ]:
ac.plot_ROCAUC_mult(y_test=[outs_pd[0]['y_test']],
                    y_score=[outs_pd[0]['risk']],
                    labels=['PD IDEARS'],
                    cols=['blue'],
                    figname='PD ROCAUC',
                    figx=6,figy=4)